通用函数

In [7]:
import re
from collections import defaultdict
from itertools import chain

first = {'ch': 'i',
         'sh': 'u',
         'zh': 'v'}

second = {
    'ua': 'w',
    'ei': 'z',
    'e': 'e',
    'ou': 'b',
    'iu': 'q',
    've': 't',
    'ue': 't',
    'u': 'u',
    'i': 'i',
    'o': 'o',
    'uo': 'o',
    'ie': 'x',
    'a': 'a',
    'ong': 's',
    'iong': 's',
    'ai': 'l',
    'ing': 'y',
    'uai': 'y',
    'ang': 'h',
    'uan': 'r',
    'an': 'j',
    'en': 'f',
    'ia': 'w',
    'iang': 'd',
    'uang': 'd',
    'eng': 'g',
    'in': 'n',
    'ao': 'k',
    'v': 'v',
    'ui': 'v',
    'un': 'p',
    'iao': 'c',
    'ian': 'm'
}

# 特殊，只有䪨母，且总长不过 3
# 零声母，单双三䪨母
special = {
    'a': 'aa',
    'ai': 'ai',
    'an': 'an',
    'ang': 'ah',
    'ao': 'ao',
    'e': 'ee',
    'ei': 'ei',
    'en': 'en',
    'er': 'er',
    'o': 'oo',
    'ou': 'ou',

    # 唵嘛呢叭咪吽
    'ong': 'os',
    # 嗯
    'eng': 'eg',
}

def to_double(s: str) -> str:
    """
    传入单汉字的全拼编码，反回其自然码双拼编码

    :param s: 全拼编码
    :return: 双拼编码
    """
    new_s = ''
    # 特列情况: 无声母，a, an, ang, n
    if len(s) <= 3 and s[0] in ['a', 'e', 'o']:
        if s in special.keys():
            return special[s]
        else:
            print('未知情况1', s)

    # 一般: 声母 + 䪨母

    # 最长的情况：first+second，例如 chuang = ch + uang
    # 2 位声母 + 最多 4 位韵母
    if s[:2] in first.keys():
        new_s += first[s[:2]]
        # 最多 4 位䪨母
        if s[2:] in second.keys():
            new_s += second[s[2:]]
    # 较短的情况：second+second，例如 h uang, x iang
    # 1位声母 + 最多 4 位䪨母
    else:
        new_s += s[0]  # 1位声母
        # 最多 4 位䪨母
        if s[1:] in second.keys():
            new_s += second[s[1:]]
        else:
            new_s += s[1:]

    return new_s


pattern = re.compile(r'^[a-zA-Z"]+$')
def is_en(text):
    return bool(pattern.match(text))

# 中文分成每一个字，英文分成一个单词
def split_zh_en(text):
    pattern = re.compile(r'[^a-zA-Z]|[a-zA-Z]+')
    return pattern.findall(text)

构造自然码+形码码表

In [8]:
# 是否使用41448大字表
enable_40k = False

standard = set()

with open('dicts/8105.dict.yaml', encoding='utf-8') as f:
    for line in f:
        line = line.rstrip()
        if re.match(r'^.\t\w+\t\d+$', line):
            char, _, freq = line.split()
            standard.add(char)

if enable_40k:
    with open('dicts/41448.dict.yaml', encoding='utf-8') as f:
        for line in f:
            line = line.rstrip()
            if re.match(r'^.\t\w+\t\d+$', line):
                char, _, freq = line.split()
                if char not in standard:
                    standard.add(char)

# 鹤形
# xcodes = defaultdict(set)
# with open('./dicts/flypy_flypy.dict.yaml', 'r', encoding='utf-8') as f:
#     for line in f:
#         line = line.rstrip()
#         if re.match(r'^.\t[\w\[]+.*$', line):
#             char, code = line.split('\t')[:2]
#             if char not in standard:
#                 continue
#             code = code[code.rfind('[')+1:]
#             if code:
#                 xcodes[char].add(code)

# 键道6
parts = {
    '一': 'a', # jd6默认 v
    '贝': 'b', # jd6默认 ao
    '草': 'c', # jd6默认 ii
    '丶': 'd',
    # e
    '手': 'f', # jd6默认 iu
    # g
    # i=ch
    '金': 'j', # jd6默认 io
    '口': 'k', # jd6默认 o
    '丨': 'l',
    '木': 'm', # jd6默认 v
    '日': 'o', # jd6默认 oi
    '丿': 'p',
    # q
    '人': 'r', # jd6默认 r
    '土': 't', # jd6默认 vo
    '士': 't', # jd6默认 vo
    # u=sh
    '水': 'u', # jd6默认 a
    '十': 'x', # jd6默认 ui
    '乛': 'v',
    '月': 'y', # jd6默认 u
}

def get_chaizi(path):
    with open(path, 'r', encoding='utf-8') as f:
        chaizi = {}
        for line in f:
            line = line.rstrip()
            if not line or line.startswith('#'):
                continue
            char, parts = line.split('\t')[:2]
            if char not in standard or parts == '未知':
                continue
            chaizi[char] = parts
        return chaizi

chaizi_jd6 = get_chaizi('./dicts/chaizi_jd6.txt')
chaizi_jd6['搬'] = "手丿丿乛"
xcodes = {key: [''.join([parts[c] if len(parts[c]) == 1 else to_double(parts[c]) for c in value])] for key, value in chaizi_jd6.items()}

missing = set()
multiple = set()
predefined = """---
name: zrm
version: "1.0"
sort: by_weight  
use_preset_vocabulary: true
min_phrase_weight: 100
import_tables:
  - ./dicts/custom
  - ./dicts/base
  - ./dicts/ext
  - ./dicts/fix
...

"""

with open('../zrm.dict.yaml', 'w', encoding='utf-8') as f1, open('dicts/8105.dict.yaml', encoding='utf-8') as f2, open('dicts/41448.dict.yaml', encoding='utf-8') as f3:
    f1.write(predefined)
    lines = chain(f2, f3) if enable_40k else f2
    for line in lines:
        line = line.rstrip()
        if re.match(r'^.\t\w+\t\d+$', line):
            char, pinyin, freq = line.split()
            if char in xcodes:
                if len(xcodes[char]) > 1:
                    multiple.add(char)
                for code in xcodes[char]:
                    f1.write(f"{char}\t{to_double(pinyin)};{code}\t{freq}\n")
            else:
                missing.add(char)
                f1.write(f"{char}\t{to_double(pinyin)};\t{freq}\n")

print(f"{len(missing)}个字的辅助码缺失：{missing}")
print(f"{len(multiple)}个字存在多个辅助码：{multiple}")

1157个字的辅助码缺失：{'藟', '冮', '溠', '鞳', '㻬', '𤭢', '俙', '詟', '𫓶', '硍', '𬹼', '幖', '酅', '囷', '媂', '飐', '𰻝', '漼', '䏲', '愭', '縢', '垿', '𫠜', '㳘', '槚', '瑢', '栻', '玭', '玒', '玘', '潟', '簕', '𣲗', '拤', '祂', '琈', '鹖', '𬘬', '𫔎', '媓', '杄', '砄', '墣', '𬘫', '瑅', '佁', '龂', '𨐈', '袯', '倕', '硚', '萩', '牥', '嶓', '玓', '晅', '刬', '藦', '挓', '漦', '慆', '䓬', '螓', '墐', '甦', '侹', '哱', '澂', '牠', '涐', '堾', '矼', '佧', '欂', '嬬', '瑝', '漴', '𬤝', '摏', '𬳽', '徛', '鄜', '鹲', '𬭎', '亸', '𬭛', '飗', '濋', '泃', '勠', '帨', '坥', '脩', '葴', '韂', '嚄', '稌', '脿', '瑄', '伋', '冔', '䞍', '屾', '筤', '狉', '埫', '嵅', '琔', '㤘', '㕮', '䗪', '䓛', '瑔', '矻', '趯', '堧', '穙', '菉', '儳', '𬭸', '儴', '𫶇', '翷', '𬬮', '唝', '麖', '墘', '侴', '暵', '䴘', '玿', '砠', '𪨰', '𫖳', '暕', '𬶋', '𫑡', '𫖮', '𨟠', '剟', '𫠊', '翯', '琯', '𬊤', '崡', '薸', '蓂', '𬬱', '甡', '靿', '媆', '𬮱', '𬭚', '溚', '偡', '跩', '煓', '棬', '珒', '嶍', '鄚', '𩾃', '㭎', '𬒈', '𫍲', '玙', '揕', '𥔲', '㬚', '艅', '匼', '豮', '夐', '薳', '郪', '腯', '挼', '穜', '旿', '碨', '朓', '𠳐', '嬢', '焌', '𫍯', '玼', '𫟹', '胠', '斶', '濩', '𫢸', '峧', '荄', '𫌀', '硁', '䂮', '滍', '

重码

In [3]:
code2chars = defaultdict(set)
charcode2freq = dict()

with open('../zrm.dict.yaml', encoding='utf-8') as f:
    for line in f:
        line = line.rstrip()
        if re.match(r'^.\t[a-z;]+\t\d+$', line):
            char, code, freq = line.split()
            code2chars[code].add(char)
            charcode2freq[char + code] = int(freq)

with open('../zrm.dict.yaml', 'r', encoding='utf-8') as file:
    lines = file.readlines()

with open('../zrm.dict.yaml', 'r', encoding='utf-8') as file:
    for line in lines:
        line = line.rstrip()
        if re.match(r'^.\t[a-z;]+\t\d+$', line):
            char, code, freq = line.split()
            if len(code2chars[code]) > 1:
                print(code2chars[code])
        #         fs = [charcode2freq[c + code] for c in code2chars[code]]
        #         fs.sort(reverse=True)
        #         code = code + fs.index(int(freq)) * 'j'
        #         line = '\t'.join([char, code, freq])
        # file.write(line + '\n')

{'熜', '蟌'}
{'憺', '𬘘', '丼', '𫢸', '疍', '鿕'}
{'祋', '㨃'}
{'砆', '㕮', '茀', '洑', '𫓧', '芣', '琈', '韨', '𫖯', '榑', '垺', '簠', '鄜', '鳆', '㳇', '玞'}
{'鿔', '滆'}
{'㬊', '澴', '荁', '𬘫', '睆', '峘', '貆', '𤩽'}
{'㴔', '耤', '𬶨', '罽', '惎', '堲', '徛', '𪟝', '𬯀', '伋', '蹐', '檵', '𫓯', '姞', '瀱', '𫌀', '穄', '枅', '𫓹', '觭', '𬶭', '垍', '漈'}
{'朘', '焆'}
{'圐', '矻', '堀'}
{'㰀', '珕', '醨', '浬', '𫟷', '凓', '𫵷', '𬍛', '浰', '朸'}
{'蓂', '醾', '洣', '蒾', '祕'}
{'𧿹', '霂', '𬭁'}
{'𫐐', '鿭', '薿', '𨺙', '𫠜', '麑'}
{'㖏', '嵲', '糵', '菍', '𫔶'}
{'𫪘', '𬉼', '𠙶'}
{'舥', '潖', '𤆵'}
{'珮', '衃'}
{'玭', '𬞟', '榀'}
{'骎', '螓', '梣'}
{'碃', '庼', '䞍', '勍'}
{'𬒈', '卻', '碏', '埆'}
{'朊', '媆', '瓀', '堧'}
{'媞', '襫', '辻', '奭', '祏', '䏡', '䴓', '𬤊', '栻', '鼫', '浉', '𫚕'}
{'凘', '涘', '飔', '亖', '㟃', '楒', '虒'}
{'鞳', '祂', '褟', '鿎', '阘', '牠'}
{'䏲', '媞', '𫘨', '䴘', '瑅', '趯', '擿', '薙', '遆'}
{'湉', '沺', '瑱', '鿬', '畑', '黇', '盷', '淟', '晪'}
{'湉', '沺', '瑱', '鿬', '畑', '黇', '盷', '淟', '晪'}
{'曈', '橦', '𦒍', '㛚', '烔', '𫍣', '樋', '峂', '穜', '哃'}
{'凃', '㻬', '稌', '葖', '梌', '𬳿', '腯'}
{'焞', '忳', '坉', '氽'}
{'𫘜', '炆'

词库转换准备

In [9]:
zrm_phones = defaultdict(list)
zrm_codes = {}
zrm_path = '../zrm.dict.yaml'

with open(zrm_path, encoding='utf-8') as f:
  for line in f:
    if len(line.split('\t')) != 3:
      continue
    char, code = line.rstrip().split('\t')[:2]
    yin, xing = code.split(';')
    zrm_phones[char].append(yin)
    zrm_codes[char] = xing

def convert_pinyin(infile, outfile, override_freq=None):
  with open(infile, encoding='utf-8') as f1, open(outfile, 'w', encoding='utf-8') as f2:
    for line in f1:
      if not re.match(r'^.+\t.+(\t\d+)*$', line):
        f2.write(line)
        continue
      parts = line.rstrip().split('\t')
      chars, phones = split_zh_en(parts[0]), parts[1].split(' ')
      if len(chars) < 1:  # 排除单字
        continue

      if any(is_en(char) for char in chars): # 存在英语
        doubled = (phone if is_en(char) else to_double(phone) for char,phone in zip(chars, phones)) # 存在英语的时候加形码会导致输入框提示异常
        doubled_code = ''.join(doubled)
      elif all((char in zrm_phones and to_double(phone) in zrm_phones[char]) for char,phone in zip(chars, phones)):
        doubled = (phone if is_en(char) else to_double(phone) + ';' + zrm_codes[char] for char,phone in zip(chars, phones))
        # doubled = (phone if is_en(char) else to_double(phone) for char,phone in zip(chars, phones))
        doubled_code = ' '.join(doubled)
      else: # 存在词库读音与字库设置读音不符
        continue

      if len(parts) < 3:
        if override_freq:
          f2.write(''.join(chars) + '\t' + doubled_code + '\t' + str(override_freq) + '\n')
        else:
          f2.write(''.join(chars) + '\t' + doubled_code + '\n')
      else:
        freq = parts[2]
        f2.write(''.join(chars) + '\t' + doubled_code + '\t' + freq + '\n')

自定义词库转换

In [10]:
convert_pinyin('./dicts/custom.dict.yaml', '../dicts/custom.dict.yaml', override_freq=10000)
convert_pinyin('./dicts/fix.dict.yaml', '../dicts/fix.dict.yaml')

其他词库转换

In [6]:
convert_pinyin('./dicts/base.dict.yaml', '../dicts/base.dict.yaml')
convert_pinyin('./dicts/ext.dict.yaml', '../dicts/ext.dict.yaml')